In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hatchet
from hatchet.readers.gptl_reader import GPTLReader
import pandas as pd
from hatchet.frame import Frame
from hatchet.node import Node

In [3]:
file_location = '../../../hatchet/tests/data/gptl/log_cpu'
dir_location = '../../../hatchet/tests/data/gptl/multi-node/'

In [25]:
gptlr = GPTLReader(file_location)
gptlr.read_log_dir(dir_location)

['../../../hatchet/tests/data/gptl/multi-node/model_timing.0000', '../../../hatchet/tests/data/gptl/multi-node/model_timing.2048', '../../../hatchet/tests/data/gptl/multi-node/model_timing.2752']
[0, 2048, 2752, 3456]
../../../hatchet/tests/data/gptl/multi-node/model_timing.0000
['On', 'Called Recurse', 'Wallclock', 'max', 'min', 'UTR Overhead']
0
line CPL:INIT"                                                                   -       1    -     120.762016   120.762016   120.762016         0.000000 

split line init ['CPL:INIT', '                                                                   -       1    -     120.762016   120.762016   120.762016         0.000000 \n']
col names ['On', 'Called Recurse', 'Wallclock', 'max', 'min', 'UTR Overhead']
split line ['-', '1', '-', '120.762016', '120.762016', '120.762016', '0.000000']
node dict {'depth': 0, 'name': 'CPL:INIT', 'On': '-', 'Called Recurse': '1', 'Wallclock': '-', 'max': '120.762016', 'min': '120.762016', 'UTR Overhead': '120.76

In [26]:

gptlr = GPTLReader(file_location)
gf = gptlr.read_log_simple(file_location)

FileNotFoundError: [Errno 2] No such file or directory: '../../../hatchet/tests/data/gptl-cpu/log_cpu'

In [ ]:
print(gf.tree(metric_column='time'))